In [1]:
import s3fs 
import xarray as xr
		
s3 = s3fs.S3FileSystem(anon=True)
def lookup(path):
    return s3fs.S3Map(path, s3=s3)
		
path = "hrrrzarr/sfc/20250705/20250705_00z_anl.zarr" 
ds = xr.open_mfdataset([lookup(path)], engine="zarr") 


In [3]:
import xarray as xr

ds = xr.open_zarr("https://data.dynamical.org/noaa/gfs/analysis-hourly/latest.zarr?email=optional@email.com")

In [4]:
ds

<xarray.Dataset> Size: 1TB
Dimensions:                (latitude: 721, longitude: 1440, time: 82920)
Coordinates:
  * latitude               (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude              (longitude) float64 12kB -180.0 -179.8 ... 179.8
    spatial_ref            int64 8B ...
  * time                   (time) datetime64[ns] 663kB 2015-01-15 ... 2024-06...
Data variables:
    precipitation_surface  (time, latitude, longitude) float32 344GB dask.array<chunksize=(160, 145, 144), meta=np.ndarray>
    temperature_2m         (time, latitude, longitude) float32 344GB dask.array<chunksize=(160, 145, 144), meta=np.ndarray>
    wind_u_10m             (time, latitude, longitude) float32 344GB dask.array<chunksize=(160, 145, 144), meta=np.ndarray>
    wind_v_10m             (time, latitude, longitude) float32 344GB dask.array<chunksize=(160, 145, 144), meta=np.ndarray>
Attributes:
    attribution:         NOAA NCEP GFS data processed by dynamical.org from N...
    description:         Historical weather data from the Global Forecast Sys...
    id:                  noaa-gfs-analysis-hourly
    name:                NOAA GFS analysis, hourly
    spatial_domain:      Global
    spatial_resolution:  0.25 degrees (~20km)
    time_domain:         2015-01-15 00:00:00 UTC to 2024-07-01 00:00:00 UTC
    time_resolution:     1 hour

In [2]:
#Checking if the files are available
import s3fs
import os

fs = s3fs.S3FileSystem(anon=True)

for day in ['03', '04', '05']:
    path = f'noaa-mrms-pds/CONUS/RadarOnly_QPE_01H_00.00/202507{day}/'
    try:
        files = fs.ls(path)
        print(f"{day}: Found {len(files)} files")
    except FileNotFoundError:
        print(f"{day}: Not found")

03: Found 718 files
04: Found 717 files
05: Found 718 files


In [ ]:
# Initialize anonymous S3 filesystem
fs = s3fs.S3FileSystem(anon=True)

# Output folder (make sure it exists)
output_dir = '/Users/rm65238/Documents/Work/MRMS texas'
os.makedirs(output_dir, exist_ok=True)

# Dates to download (July 3 to July 5, 2025)
dates = ['03', '04', '05']

for day in dates:
    s3_path = f'noaa-mrms-pds/CONUS/RadarOnly_QPE_01H_00.00/202507{day}/'
    try:
        files = fs.ls(s3_path)
        print(f"{day}: Found {len(files)} files")
        for file in files:
            filename = os.path.basename(file)
            local_path = os.path.join(output_dir, filename)

            # Skip if already downloaded
            if os.path.exists(local_path):
                print(f"Skipping (exists): {filename}")
                continue

            print(f"Downloading: {filename} ...")
            with fs.open(file, 'rb') as remote, open(local_path, 'wb') as local_file:
                local_file.write(remote.read())
        print(f"Finished downloading files for 2025-07-{day}\n")
    except FileNotFoundError:
        print(f"Folder not found for date 2025-07-{day}")


In [ ]:
# uncompressing the files
import gzip
import shutil
import os

folder = '/Users/rm65238/Documents/Work/MRMS texas'

for filename in os.listdir(folder):
    if filename.endswith('.gz'):
        gz_path = os.path.join(folder, filename)
        out_path = gz_path[:-3]  # remove '.gz'

        # Skip if already uncompressed
        if os.path.exists(out_path):
            print(f'Skipping already decompressed: {out_path}')
            continue

        print(f'Decompressing: {filename}')
        with gzip.open(gz_path, 'rb') as f_in, open(out_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [ ]:
import xarray as xr

file_path = '/Users/rm65238/Documents/Work/MRMS texas/MRMS_RadarOnly_QPE_01H_00.00_20250703-000000.grib2'
ds = xr.open_dataset(file_path, engine='cfgrib')
ds

/opt/miniconda3/envs/era5env/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/opt/miniconda3/envs/era5env/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/opt/miniconda3/envs/era5env/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, 

ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [4]:
ds

<xarray.Dataset> Size: 98MB
Dimensions:         (latitude: 3500, longitude: 7000)
Coordinates:
    time            datetime64[ns] 8B ...
    step            timedelta64[ns] 8B ...
    heightAboveSea  float64 8B ...
  * latitude        (latitude) float64 28kB 54.99 54.98 54.98 ... 20.02 20.01
  * longitude       (longitude) float64 56kB 230.0 230.0 230.0 ... 300.0 300.0
    valid_time      datetime64[ns] 8B ...
Data variables:
    unknown         (latitude, longitude) float32 98MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  US NOAA Office of Oceanic and Atmospheric Research
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US NOAA Office of Oceanic and Atmospheric Research
    history:                 2025-07-18T11:58 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
import xarray as xr
import glob
import os

folder = '/Users/rm65238/Documents/Work/MRMS texas'

# Find all decompressed grib2 files
file_paths = sorted(glob.glob(os.path.join(folder, '*.grib2')))

# Open all datasets and combine along the new 'time' dimension
datasets = [xr.open_dataset(fp, engine='cfgrib') for fp in file_paths]

# Concatenate all datasets along time dimension
combined_ds = xr.concat(datasets, dim='time')

print(combined_ds)

/opt/miniconda3/envs/era5env/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/opt/miniconda3/envs/era5env/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/opt/miniconda3/envs/era5env/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, 